In [ ]:
import random
import math
import time

# Constants
EMPTY, RED, BLUE = 0, 1, 2
QUADRANT_SIZE = 3
BOARD_SIZE = 6

# Initialize the board
def init_board():
    return [[EMPTY] * BOARD_SIZE for _ in range(BOARD_SIZE)]

# Print the board
def print_board(board):
    symbols = {EMPTY: '.', RED: 'R', BLUE: 'B'}
    for row in board:
        print(' '.join(symbols[cell] for cell in row))
    print()

# Check for a win condition
def check_winner(board, player):
    # Check rows, columns, and diagonals
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE - 4):
            if all(board[r][c + i] == player for i in range(5)):
                return True
    for c in range(BOARD_SIZE):
        for r in range(BOARD_SIZE - 4):
            if all(board[r + i][c] == player for i in range(5)):
                return True
    for r in range(BOARD_SIZE - 4):
        for c in range(BOARD_SIZE - 4):
            if all(board[r + i][c + i] == player for i in range(5)):
                return True
            if all(board[r + i][c + 4 - i] == player for i in range(5)):
                return True
    return False

# Place two pieces on the board
def place_pieces(board, row1, col1, row2, col2, player):
    if board[row1][col1] == EMPTY and board[row2][col2] == EMPTY:
        new_board = [row[:] for row in board]
        new_board[row1][col1] = player
        new_board[row2][col2] = player
        return new_board
    return None

# Rotate a quadrant
def rotate_quadrant(board, quadrant, direction):
    start_row = (quadrant - 1) // 2 * QUADRANT_SIZE
    start_col = (quadrant - 1) % 2 * QUADRANT_SIZE
    new_board = [row[:] for row in board]

    for r in range(QUADRANT_SIZE):
        for c in range(QUADRANT_SIZE):
            if direction == '+':
                new_r, new_c = c, QUADRANT_SIZE - 1 - r
            else:
                new_r, new_c = QUADRANT_SIZE - 1 - c, r
            new_board[start_row + new_r][start_col + new_c] = board[start_row + r][start_col + c]

    return new_board

# Minimax Algorithm with Alpha-Beta Pruning
def minimax(board, depth, alpha, beta, is_maximizing, player, start_time, max_time):
    if time.time() - start_time > max_time:
        return 0  # Timeout safeguard
    
    opponent = RED if player == BLUE else BLUE

    if depth == 0 or check_winner(board, RED) or check_winner(board, BLUE):
        if check_winner(board, player):
            return 10
        elif check_winner(board, opponent):
            return -10
        else:
            return 0

    if is_maximizing:
        max_eval = -math.inf
        for row1 in range(BOARD_SIZE):
            for col1 in range(BOARD_SIZE):
                if board[row1][col1] == EMPTY:
                    for row2 in range(BOARD_SIZE):
                        for col2 in range(BOARD_SIZE):
                            if (row1 != row2 or col1 != col2) and board[row2][col2] == EMPTY:
                                for quad in range(1, 5):
                                    for dir in ['+', '-']:
                                        temp_board = place_pieces(board, row1, col1, row2, col2, player)
                                        if temp_board:
                                            new_board = rotate_quadrant(temp_board, quad, dir)
                                            eval = minimax(new_board, depth - 1, alpha, beta, False, player, start_time, max_time)
                                            max_eval = max(max_eval, eval)
                                            alpha = max(alpha, eval)
                                            if beta <= alpha:
                                                break
        return max_eval
    else:
        min_eval = math.inf
        for row1 in range(BOARD_SIZE):
            for col1 in range(BOARD_SIZE):
                if board[row1][col1] == EMPTY:
                    for row2 in range(BOARD_SIZE):
                        for col2 in range(BOARD_SIZE):
                            if (row1 != row2 or col1 != col2) and board[row2][col2] == EMPTY:
                                for quad in range(1, 5):
                                    for dir in ['+', '-']:
                                        temp_board = place_pieces(board, row1, col1, row2, col2, opponent)
                                        if temp_board:
                                            new_board = rotate_quadrant(temp_board, quad, dir)
                                            eval = minimax(new_board, depth - 1, alpha, beta, True, player, start_time, max_time)
                                            min_eval = min(min_eval, eval)
                                            beta = min(beta, eval)
                                            if beta <= alpha:
                                                break
        return min_eval

# Get best move for the AI
def get_best_move(board, depth, player, max_time=5):
    best_move = None
    best_val = -math.inf
    start_time = time.time()

    for row1 in range(BOARD_SIZE):
        for col1 in range(BOARD_SIZE):
            if board[row1][col1] == EMPTY:
                for row2 in range(BOARD_SIZE):
                    for col2 in range(BOARD_SIZE):
                        if (row1 != row2 or col1 != col2) and board[row2][col2] == EMPTY:
                            for quad in range(1, 5):
                                for dir in ['+', '-']:
                                    temp_board = place_pieces(board, row1, col1, row2, col2, player)
                                    if temp_board:
                                        new_board = rotate_quadrant(temp_board, quad, dir)
                                        move_val = minimax(new_board, depth - 1, -math.inf, math.inf, False, player, start_time, max_time)
                                        if move_val > best_val:
                                            best_val = move_val
                                            best_move = (row1, col1, row2, col2, quad, dir)
                                        if time.time() - start_time > max_time:
                                            return best_move  # Timeout safeguard
    return best_move

# Play game
def play_game():
    board = init_board()
    players = [RED, BLUE]
    current_player = 0

    while True:
        print_board(board)
        if current_player == 0:
            # Player's move
            move = input("Enter your move (row1 col1 row2 col2 quadrant direction): ").strip()
            if len(move) < 6:
                print("Invalid move")
                continue
            row1, col1, row2, col2, quad, direction = int(move[0]), int(move[1]), int(move[2]), int(move[3]), int(move[4]), move[5]
            new_board = place_pieces(board, row1, col1, row2, col2, RED)
            if new_board:
                board = rotate_quadrant(new_board, quad, direction)
            else:
                print("Invalid move")
                continue
        else:
            # AI's move
            print("AI is thinking...")
            move = get_best_move(board, 3, BLUE)
            if move:
                row1, col1, row2, col2, quad, direction = move
                new_board = place_pieces(board, row1, col1, row2, col2, BLUE)
                if new_board:
                    board = rotate_quadrant(new_board, quad, direction)
                print(f"AI's move: {row1} {col1} {row2} {col2} {quad} {direction}")

        if check_winner(board, RED):
            print_board(board)
            print("Red wins!")
            break
        if check_winner(board, BLUE):
            print_board(board)
            print("Blue wins!")
            break
        if all(board[row][col] != EMPTY for row in range(BOARD_SIZE) for col in range(BOARD_SIZE)):
            print_board(board)
            print("It's a draw!")
            break

        current_player = 1 - current_player

if __name__ == "__main__":
    play_game()


. . . . . .
. . . . . .
. . . . . .
. . . . . .
. . . . . .
. . . . . .



Enter your move (row1 col1 row2 col2 quadrant direction):  22334+


. . . . . .
. . . . . .
. . R . . .
. . . . . R
. . . . . .
. . . . . .

AI is thinking...
